In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

For large datasets, it is better to use shell commands to retrieve the train database. 

In [2]:
## check if GPU is available and use it; otherwise use CPU
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

In [ ]:
%%time
sub_df= pd.read_csv('sample_submission.csv')
test_df= pd.read_csv('test.csv')
df= pd.read_csv('input/train.csv')
df.sample(5)

In [4]:
data_dir = '../input/new-york-city-taxi-fare-prediction'
!ls -lh {data_dir}

In [5]:
%%time
!wc -l {data_dir}/train.csv

In [6]:
%%time
!wc -l {data_dir}/test.csv

In [7]:
%%time
!wc -l {data_dir}/sample_submission.csv

In [8]:
%%time
# Training set
!head {data_dir}/train.csv

In [9]:
%%time
# Test set
!head {data_dir}/test.csv

In [10]:
%%time
#  Sample sub
!head {data_dir}/sample_submission.csv

In [11]:
%%time
# Importing the data in pieces

df_test = pd.read_csv(data_dir+'/test.csv',parse_dates=['pickup_datetime'])
df_test

In [12]:
%%time
df_test.shape

In [13]:
df_test.key.nunique()

loading the Training data

In [14]:
%%time
import random # selecting random satasets from training data sets
sample_frac = 0.20  #loading only 20% data at once

selected_cols = 'fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count'.split(',')
dtypes = {
    'fare_amount': 'float16', 
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'passenger_count': 'uint8'
}

## this function will return Treu for (1- sample_frac) thus these rows will be skipped 

def skip_row(row_idx):
    if row_idx == 0:
        return False
    return random.random() > sample_frac 

random.seed(7)
df = pd.read_csv(data_dir+"/train.csv", 
                 usecols=selected_cols, 
                 dtype=dtypes, 
                 parse_dates=['pickup_datetime'], 
                 skiprows=skip_row)
df_original =df.copy()
df

In [57]:
df =df_original.copy()
df

In [58]:
%%time
df.info()

Cleaning and exploring the dataset

In [59]:
%%time
#  checking for null values in dataset 
df.isnull().sum()

In [60]:
%%time
df = df.dropna() # drop null values found at earlier stage 

In [61]:
%%time
df.isnull().sum()

In [62]:
%%time
#### Checking for duplicates
df.duplicated().sum()

In [63]:
%%time
# There are 42 duplicates, Lets remove them
df = df.drop_duplicates()  ## dropes duplicates
df.duplicated().sum()

In [64]:
df.info()

In [65]:
%%time
df.describe()

# 3. Feature Engineering
After some exploraation I realised I sholud perform **Feature Engineering** to understand the data properly  
as I saw fare to be -ve in around 463 rows and fare value more than 500 few time and also infinite twice or trice
1. add a feature for distance between pickup place and drop place
2. Need to perform feature extraction for time

In [66]:
%%time
# Add haversine distance (great circle distance between two points on earth given teir longitude and lattitude)

import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [67]:
%%time
def add_trip_distance(df):
    df['trip_distance'] = haversine_np(df['pickup_longitude'], df['pickup_latitude'], df['dropoff_longitude'], df['dropoff_latitude'])

In [68]:
%%time
add_trip_distance(df)

In [69]:
def add_dateparts(df, col):
    df['year'] = df[col].dt.year
    df['month'] = df[col].dt.month
    df['day'] = df[col].dt.day
    df['weekday'] = df[col].dt.weekday
    df[col + '_hour'] = df[col].dt.hour

In [70]:
%%time
add_dateparts(df, 'pickup_datetime')

In [71]:
%%time
df.head()

# 2. **EDA** cont...

I wanted to see relationship between -ve fare value with distance thus decided to perform feature engineering before hand 

## To check anamolies in the data

In [72]:
df

In [74]:
dt1 = df.loc[df['fare_amount']  < 0 ]
dt1

In [76]:
dt1.describe()

#### Fare amount
In most of the cases where fare amountis negative, the following are the observation: 
* Pickup location is same as the drop loacation 
* either the pickup longitude/lattitude or dropoff longitude/lattitude is 0


In [78]:
## Checking for fare value to be greater than 500
dt2 = df[df['fare_amount']>500]
dt2

In [79]:
dt2.describe()

#### Fare amount > 500
In most of the cases where fare amountis negative, the following are the observation: 
* Pickup location is same as the drop loacation 
* either the pickup longitude/lattitude or dropoff longitude/lattitude is 0
* Maximum value of amount is infinity 
* only 10 rows are found to be > 500, definitely an outlier


In [80]:
df[df['pickup_longitude']==0]

In [81]:
## Lets check for distance = 0
df[df['trip_distance']==0]

In [82]:
dt = df[df['passenger_count']>6]
dt

In [83]:
df.pickup_datetime.min(), df.pickup_datetime.max()

In [37]:
%%time
df = df.drop('pickup_datetime', axis=1)
df.head(2)

In [38]:
df.hist(figsize=(22,21), bins=20);

# 3.1 Removing Outliers
We'll use the following ranges:

* fare_amount: 1 to 500
* longitudes: -75 to -72
* latitudes: 40 to 42
* passenger_count: 1 to 6

In [84]:
%%time
def remove_outliers(df):
    return df[(df['fare_amount'] >= 1.) & 
              (df['fare_amount'] <= 500.) &
              (df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 6)]

In [85]:
%%time
df = remove_outliers(df)

# Ask & answer questions about the dataset: 

1. What is the busiest day of the week?
2. What is the busiest time of the day?
3. In which month are fares the highest?
4. Which pickup locations have the highest fares?
5. Which drop locations have the highest fares?
6. What is the average ride distance?

EDA + asking questions will help you develop a deeper understand of the data and give you ideas for feature engineering.

In [86]:
# 1. What is the busiest day of the week?
df.weekday.mode()

In [87]:
# 2. What is the busiest time of the day?
df.pickup_datetime_hour.mode()

In [88]:
# 3. In which month are fares the highest? >>> winters have high fare and Jan has highest fare
df_fare= df.sort_values(ascending=False, by= 'fare_amount').head(50)
df_fare.month.hist()

In [43]:
# 4. Which pickup locations have the highest fares?
sns.scatterplot(x='pickup_longitude', y= 'pickup_latitude', hue='fare_amount',data=df_fare);

In [44]:
# 5. Which drop locations have the highest fares?
sns.scatterplot(x='dropoff_longitude', y= 'dropoff_latitude',hue='fare_amount',data=df_fare);

In [89]:
### Plotting log , lat for pickup
sns.scatterplot(x='pickup_longitude', y= 'pickup_latitude', data=df, hue='fare_amount')

In [47]:
### Plotting log , lat for pickup
sns.scatterplot(x='dropoff_longitude', y= 'dropoff_latitude', data=df , hue='fare_amount')

### FOr test dataset

In [91]:
%%time
df_test.hist(figsize=(8,7), bins=20)

In [92]:
%%time
df_test[df_test['pickup_longitude']==0]

In [93]:
df_test[df_test['passenger_count']>6]

In [90]:
df_test.pickup_datetime.min(), df_test.pickup_datetime.max()

No anamolies in test data set 

## 4. Prepare Dataset for Training

- Split Training & Validation Set
- Fill/Remove Missing Values
- Extract Inputs & Outputs
   - Training
   - Validation
   - Test

In [94]:
%%time
df.corr()

In [95]:
df.describe()

In [96]:
import seaborn as sns
#%%time
#sns.lineplot(y='fare_amount', x='pickup_datetime', data= df_original)

### Split Training & Validation Set

We'll set aside 20% of the training data as the validation set, to evaluate the models we train on previously unseen data. 

Since the test set and training set have the same date ranges, we can pick a random 20% fraction.

In [97]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(val_df)

In [98]:
df.columns

In [101]:
input_cols = ['pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'trip_distance', 'year', 'month', 'day', 'weekday',
       'pickup_datetime_hour']
target_col = 'fare_amount'

In [102]:
train_inputs = train_df[input_cols]
train_targets = train_df[target_col]
train_inputs.head(3)

In [103]:
train_targets.head(3)

Validation of training dataset on validation set 

In [104]:
val_inputs = val_df[input_cols]
val_targets = val_df[target_col]

display(val_inputs.head(3))
display(val_targets.head(3))

In [105]:
### Feature enigeerning on Test dataset
add_dateparts(df_test, 'pickup_datetime')
add_trip_distance(df_test)

# for name, lonlat in [('jfk', jfk_lonlat), ('lga', lga_lonlat), ('ewr', ewr_lonlat), ('met', met_lonlat), ('wtc', wtc_lonlat)]:
#     add_landmark_dropoff_distance(df_test, name, lonlat)
df_test.head(2)

In [106]:
test_inputs = df_test[input_cols]
test_inputs.head(3)

# 5 Modelling

## 5.1. Train Hardcoded & Baseline Models

- Hardcoded model: always predict average fare
- Baseline model: Linear regression 

For evaluation the dataset uses RMSE error: 
https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/overview/evaluation

### Train & Evaluate Hardcoded Model

general approach is to create a simple model that always predicts the average.
But we will use **linear regression **

In [107]:
%%time
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


linreg_model = LinearRegression()
linreg_model.fit(train_inputs, train_targets)
train_preds = linreg_model.predict(train_inputs)
val_preds = linreg_model.predict(val_inputs)
train_rmse = mean_squared_error(train_targets, train_preds, squared=False)
val_rmse = mean_squared_error(val_targets, val_preds, squared=False)
print('RMSE Score on Validation data',val_rmse)
print('RMSE Score on Validation data',train_rmse)

* Rmse = 5.15384799403991 mean our prediction is off by 5.153 per prediction which is not good as **fare.median is 8.5**
* our base model isn't overfitting as validation score is similar to training set

In [108]:
df.fare_amount.median()

In [109]:
from sklearn.metrics import r2_score
r2_train= r2_score(train_targets, train_preds)
r2_val = r2_score(val_targets, val_preds)
print('R2 Score on Validation data',r2_val)
print('R2 Score on Validation data',r2_train)

## 5.2 Train & Evaluate Different Models

We'll train each of the following & submit predictions to Kaggle:

- Ridge Regression
- Random Forests
- Gradient Boosting

In [110]:
def evaluate(model):
    train_preds = model.predict(train_inputs)
    train_rmse = mean_squared_error(train_targets, train_preds, squared=False)
    val_preds = model.predict(val_inputs)
    val_rmse = mean_squared_error(val_targets, val_preds, squared=False)
    return train_rmse, val_rmse, train_preds, val_preds

In [111]:
def predict_and_submit(model, fname):
    test_preds = model.predict(test_inputs)
    sub_df = pd.read_csv(data_dir+'/sample_submission.csv')
    sub_df['fare_amount'] = test_preds
    sub_df.to_csv(fname, index=None)
    return sub_df

Ridge Regression

In [112]:
from sklearn.linear_model import Ridge
model1 = Ridge(random_state=42)

In [113]:
%%time
model1.fit(train_inputs, train_targets)

In [114]:
evaluate(model1)



In [115]:
predict_and_submit(model1, 'ridge_submission.csv')
predict_and_submit(linreg_model, 'Linear_submission.csv')

In [119]:
from sklearn.ensemble import RandomForestRegressor

In [120]:
%%time
model2 = RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=7, n_estimators=50)
model2.fit(train_inputs, train_targets)

In [ ]:
evaluate(model2)

In [ ]:
predict_and_submit(model2, 'rf_submission.csv')

In [ ]:
from xgboost import XGBRegressor
model3 = XGBRegressor(random_state=42, n_jobs=-1, objective='reg:squarederror')
model3.fit(train_inputs, train_targets)

In [ ]:
evaluate(model3)

In [ ]:
predict_and_submit(model3, 'xgb_submission.csv')

## 8. Tune Hyperparmeters

https://towardsdatascience.com/mastering-xgboost-2eb6bce6bc76


We'll train parameters for the XGBoost model. Here’s a strategy for tuning hyperparameters:

- Tune the most important/impactful hyperparameter first e.g. n_estimators

- With the best value of the first hyperparameter, tune the next most impactful hyperparameter

- And so on, keep training the next most impactful parameters with the best values for previous parameters...

- Then, go back to the top and further tune each parameter again for further marginal gains

- Hyperparameter tuning is more art than science, unfortunately. Try to get a feel for how the parameters interact with each other based on your understanding of the parameter…

Let's define a helper function for trying different hyperparameters.

In [ ]:
import matplotlib.pyplot as plt

def test_params(ModelClass, **params):
    """Trains a model with the given parameters and returns training & validation RMSE"""
    model = ModelClass(**params).fit(train_inputs, train_targets)
    train_rmse = mean_squared_error(model.predict(train_inputs), train_targets, squared=False)
    val_rmse = mean_squared_error(model.predict(val_inputs), val_targets, squared=False)
    return train_rmse, val_rmse

def test_param_and_plot(ModelClass, param_name, param_values, **other_params):
    """Trains multiple models by varying the value of param_name according to param_values"""
    train_errors, val_errors = [], [] 
    for value in param_values:
        params = dict(other_params)
        params[param_name] = value
        train_rmse, val_rmse = test_params(ModelClass, **params)
        train_errors.append(train_rmse)
        val_errors.append(val_rmse)
    
    plt.figure(figsize=(10,6))
    plt.title('Overfitting curve: ' + param_name)
    plt.plot(param_values, train_errors, 'b-o')
    plt.plot(param_values, val_errors, 'r-o')
    plt.xlabel(param_name)
    plt.ylabel('RMSE')
    plt.legend(['Training', 'Validation'])

In [ ]:
best_params = {
    'random_state': 7,
    'n_jobs': -1,
    'objective': 'reg:squarederror'
}

In [ ]:
### No of trees
test_param_and_plot(XGBRegressor, 'n_estimators', [100, 250, 500], **best_params)


In [ ]:
best_params['n_estimators'] = 250

In [ ]:
%%time 
#### Max Depth
test_param_and_plot(XGBRegressor, 'max_depth', [3, 4, 5], **best_params)

In [ ]:
best_params['max_depth'] = 5

In [ ]:
%%time
#### Learning Rate
test_param_and_plot(XGBRegressor, 'learning_rate', [0.05, 0.1, 0.25], **best_params)

In [ ]:
best_params['learning_rate'] = 0.25

In [ ]:
# Final Model Creation
xgb_model_final = XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42,
                               n_estimators=500, max_depth=5, learning_rate=0.1, 
                               subsample=0.8, colsample_bytree=0.8)

In [ ]:
%%time
xgb_model_final.fit(train_inputs, train_targets)

In [ ]:
evaluate(xgb_model_final)

In [ ]:
predict_and_submit(xgb_model_final, 'xgb_tuned_submission.csv')